In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!pip install SimpleITK
!pip install xlsxwriter

In [ ]:
import os
import SimpleITK as sitk
from keras.preprocessing.image import save_img
import cv2
from matplotlib import pyplot as plt
import numpy as np
import sys

import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.pylab as pylab

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib
from skimage import img_as_float,color
from skimage.io import imsave
from skimage.exposure import adjust_gamma
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
PATH_GT = "/content/drive/MyDrive/gt"  #path to the gt folder containing all the annotated .nii.gz files
PATH_DATA = "/content/drive/MyDrive/data" #path to the data folder containing all the raw HRCT scans of .nii.gz files

In [ ]:
fileList_GT = []
for root, dirs, files in os.walk(PATH_GT):
  for filename in files:
    fileList_GT.append(filename)

In [ ]:
fileList_DATA = []
for root, dirs, files in os.walk(PATH_DATA):
  for filename in files:
    fileList_DATA.append(filename)

In [ ]:
filePaths_GT = []
for files in fileList_GT:
  paths = os.path.join(PATH_GT,files)
  filePaths_GT.append(paths)
  

In [ ]:
filePaths_DATA = []
for files in fileList_DATA:
  paths = os.path.join(PATH_DATA,files)
  filePaths_DATA.append(paths)

In [ ]:
print(len(filePaths_DATA))

In [ ]:
def readImage_GT(path):
    inputImage = sitk.ReadImage(path)
    image = sitk.GetArrayFromImage(inputImage)

    n_z = image.shape[0]
    n_x = image.shape[1]
    n_y = image.shape[2]

    volume=np.zeros((n_z,n_x,n_y,3),dtype=np.float32)

    red_multiplier = [255, 0,0]
    blue_multiplier = [0,160,255]
    yellow_multiplier=[255,255,128]
    black_multiplier = [0,0,0]

    for i in range(n_z):
        blue = np.argwhere(image[i,:,:]==4)
        yellow = np.argwhere(image[i,:,:]==5)
        red = np.argwhere(image[i,:,:]==7)
        black = np.argwhere(image[i,:,:]==0)

        gray_img = img_as_float(image[i,:,:]/image[i,:,:].max())
        imagee=adjust_gamma(color.gray2rgb(gray_img),0.45)


        for j in range(len(blue)):
            imagee[blue[j][0]][blue[j][1]]=blue_multiplier 
        
        for j in range(len(yellow)):
            imagee[yellow[j][0]][yellow[j][1]]=yellow_multiplier 

        for j in range(len(red)):
            imagee[red[j][0]][red[j][1]]=red_multiplier


        for j in range(len(black)):
            imagee[black[j][0]][black[j][1]]=black_multiplier

        volume[i,:,:]=imagee

    return volume

In [ ]:
def readImage_DATA(path):
    inputImage = sitk.ReadImage(path)
    image = sitk.GetArrayFromImage(inputImage)
    return image

In [ ]:
PATH_PNG_GT  = os.path.join(PATH_GT,'png')
os.mkdir(PATH_PNG_GT)

In [ ]:
PATH_PNG_DATA  = os.path.join(PATH_DATA,'png')
os.mkdir(PATH_PNG_DATA)

In [ ]:
for paths in filePaths_GT:
  original_fileName=os.path.basename(paths).replace('.nii.gz','')
  PATH_PATIENT = os.path.join(PATH_PNG_GT,original_fileName)
  os.mkdir(PATH_PATIENT)
  image_seg = readImage_GT(paths)
  for j in range(image_seg.shape[0]):
    save_img(os.path.join(PATH_PATIENT,original_fileName+'_slice_'+str(j+1)+'.png'),image_seg[j,:,:])

In [ ]:
print(len(filePaths_DATA))

In [ ]:
for paths in filePaths_DATA:
  original_fileName=os.path.basename(paths).replace('.nii.gz','')
  PATH_PATIENT = os.path.join(PATH_PNG_DATA,original_fileName)
  os.mkdir(PATH_PATIENT)
  image_seg = readImage_DATA(paths)
  for j in range(image_seg.shape[0]):
    matplotlib.image.imsave(os.path.join(PATH_PATIENT,original_fileName+'_slice_'+str(j+1)+'.png'),image_seg[j,:,:],cmap = "gray")

In [ ]:
in  putImage = sitk.ReadImage(filePaths_GT[4])
image = sitk.GetArrayFromImage(inputImage)
print(image.shape)